# Extract, transform and load 

Start: 
* MongoDB database full with all 2 million + issues as defined by Montgomery et al.
  * Each repository is their own collection 
* Have Jira endpoints and credentials

End:
* MongoDB database with 2 collections that hold all from different repositories:
    * issues
    * sprints
* this means that each document needs a new field with which repository it belongs to

Steps:
* Iterate over all the issues per repository
* If has sprint field:
  * Add repo name as field
  * Get sprint info from issue
  * Check if sprint already exists, if not, add it to sprint collection
  * Add the issue to the issue collection

In [2]:
# Auxiliary
import pymongo
import re
import json

client = pymongo.MongoClient("mongodb://localhost:27017")

In [4]:
def get_customfield_from_collection(collection):
    col = db[collection]
    issues = col.find()
    for issue in issues:
        fields = issue['fields']
        for field in fields:
            field_value = fields[field]
            if type(field_value) is list and len(field_value)>0 and "com.atlassian.greenhopper.service.sprint.Sprint" in field_value[0] and "custom":
                return(field)

In [3]:
def get_and_store_sprint_from_issue(sprintstring, issueid, repo):
    d = {}
    id = re.search(r'id=(.*?),', sprintstring).group(1)
    state = re.search(r'state=(.*?),', sprintstring).group(1)
    result = client.JiraRepos['02_Sprints'].find_one({"id": repo + str(id)})
    # only create and insert if it does not already exist.
    if result is None:
        d = {}
        name = re.search(r'name=(.*?),', sprintstring).group(1)
        startDate = re.search(r'startDate=(.*?),', sprintstring).group(1)
        endDate = re.search(r'endDate=(.*?),', sprintstring).group(1)
        completeDate = re.search(r'completeDate=(.*?),', sprintstring).group(1)
        d['id'] = str(repo) + str(id)
        d['state'] = state
        d['name'] = name
        d['startDate'] = startDate
        d['endDate'] = endDate
        d['completeDate'] = completeDate
        # put in try block due to a few edge cases having no activatedDate
        try:
            activatedDate = re.search(r'activatedDate=(.*?),', sprintstring).group(1)
            d['activatedDate'] = activatedDate
        except:
            d['activatedDate'] = "not present"
        d['issues'] = [issueid]
        client.JiraRepos['02_Sprints'].insert_one(d)
    else:
        client.JiraRepos['02_Sprints'].update_one({'id': repo + str(id)}, {'$push': {'issues': issueid}})

In [5]:
# test if get_and_store_sprint_from_issue function works
sprint1 = 'com.atlassian.greenhopper.service.sprint.Sprint@352cab98[id=1255,rapidViewId=471,state=CLOSED,name=NXRM Neo Sprint 25 Holding,startDate=<null>,endDate=<null>,completeDate=<null>,activatedDate=<null>,sequence=1510,goal=<null>,autoStartStop=false]'
sprint2 = 'com.atlassian.greenhopper.service.sprint.Sprint@352cab98[id=1255,rapidViewId=471,state=CLOSED,name=NXRM Neo Sprint 25 Holding,startDate=<null>,endDate=<null>,completeDate=<null>,activatedDate=<null>,sequence=1510,goal=<null>,autoStartStop=false]'
get_and_store_sprint_from_issue(sprint1, 1)
get_and_store_sprint_from_issue(sprint2, 1)


TypeError: get_and_store_sprint_from_issue() missing 1 required positional argument: 'repo'

In [6]:
# dictionary to store results. manually added SecondLife due to differing name
d = {}

# get all issues to iterate over
db = client['JiraRepos']
collections = db.list_collection_names()

# iterate over collections
for collection in collections:
    print("Starting " + collection)
    field_name = get_customfield_from_collection(collection)
    if field_name is not None:
        d[collection] = field_name

d

Starting Mindville
Starting Jira
Starting IntelDAOS
Starting Mojang
Starting Sakai
Starting RedHat
Starting SecondLife
Starting Qt
Starting JiraEcosystem
Starting Spring
Starting MongoDB
Starting MariaDB
Starting Hyperledger
Starting Sonatype
Starting Apache
Starting JFrog


{'Jira': 'customfield_11930',
 'RedHat': 'customfield_12310940',
 'Qt': 'customfield_10302',
 'Spring': 'customfield_10480',
 'MongoDB': 'customfield_10557',
 'MariaDB': 'customfield_10400',
 'Hyperledger': 'customfield_10004',
 'Sonatype': 'customfield_11001',
 'Apache': 'customfield_12310921'}

In [7]:
db_src = client['JiraRepos']
db_dest = client['JiraRepos']

for collection in d.keys():
# for collection in ['MariaDB']:
    print("Starting " + collection)
    col_src = db_src[collection]
    col_dest = db_dest['01_Issues']
    issues = col_src.find()
    for issue in issues:
        fields = issue['fields']
        for field in fields:
            # check if issue has field that holds sprint info
            if d[collection] in field and fields[field] is not None:
                # add repo field and prepend repo in id field
                issue['repository'] = collection
                # add field on whether data is private, indirect public or public
                issue['availability'] = 'indirect'
                issue['id'] = collection+str(issue['id'])
                col_dest.insert_one(issue)  
                # check for sprint data and store
                try:
                    get_and_store_sprint_from_issue(issue['fields'][field][0], issue['id'], collection)
                except:
                    print(collection, issue['id'], issue['fields'][field])

Starting Jira
Jira Jira1528839 []
Jira Jira1475868 []
Jira Jira1264865 []
Jira Jira1165963 []
Jira Jira959966 []
Starting RedHat
RedHat RedHat12777763 []
RedHat RedHat12777260 []
RedHat RedHat12777148 []
RedHat RedHat12777124 []
RedHat RedHat12777090 []
RedHat RedHat12777008 []
RedHat RedHat12776415 []
RedHat RedHat12776374 []
RedHat RedHat12776338 []
RedHat RedHat12776192 []
RedHat RedHat12775974 []
RedHat RedHat12775815 []
RedHat RedHat12775772 []
RedHat RedHat12775739 []
RedHat RedHat12770462 []
RedHat RedHat12763831 []
RedHat RedHat12756436 []
RedHat RedHat12754463 []
RedHat RedHat12752319 []
RedHat RedHat12593032 []
RedHat RedHat12590348 []
RedHat RedHat12589832 []
RedHat RedHat12589814 []
RedHat RedHat12589800 []
RedHat RedHat12589398 []
RedHat RedHat12584656 []
RedHat RedHat12584654 []
RedHat RedHat12580922 []
RedHat RedHat12580913 []
RedHat RedHat12580878 []
RedHat RedHat12570677 []
RedHat RedHat12570626 []
RedHat RedHat12570029 []
RedHat RedHat12725500 []
RedHat RedHat12721144

In [ ]:
# Get all directly available sprint data